In [22]:
import requests
import json
from datetime import datetime
import pandas as pd
import time


# API endpoint URL
url = 'https://api.terna.it/transparency/v1.0/getactualgeneration'



# Set the API key and secret
apiKey = 'v72ftysuzm8yegdh7ghyrjck'
apiSecret = 'RYu57mcpxn'

# Generate the token using the key and secret
tokenurl = 'https://api.terna.it/transparency/oauth/accessToken'
tokendata = {
    'client_id': 'v72ftysuzm8yegdh7ghyrjck',
    'client_secret': 'RYu57mcpxn',
    'grant_type': 'client_credentials'
}
tokenheaders = {'Content-Type': 'application/x-www-form-urlencoded'}

response = requests.post(tokenurl, data=tokendata, headers=tokenheaders)
access_token = json.loads(response.content.decode('utf-8'))['access_token']

# make a loop for the dates so that it makes 60 day steps until today
datelist = pd.date_range(start="2018-01-01",end="2023-01-01", freq='60D').tolist()
datelist = [datetime.strftime(date, "%d/%m/%Y") for date in datelist]
time.sleep(1)
# empty dataframe
df = pd.DataFrame()

for i in range(len(datelist)-1):
    params = {
        "dateFrom": datelist[i],
        "dateTo": datelist[i+1]
    }
    if response.status_code == 200:
         # Extract the access token from the response
        
        # Add the access token to the request headers
        headers = {'Authorization': f'Bearer {access_token}'}
        # Send the GET request to the API endpoint with the headers and parameters
        response = requests.get(url, headers=headers, params=params)
        print(response.content)
        # Convert the response to JSON
        data = response.json()
        # make a dataframe
        df_temp = pd.DataFrame(data['actualGeneration'])
        df = df.append(df_temp)
    else:
        print('Failed to generate token' + str(response.text))
    # sleep for 1 second
    time.sleep(1)
print(df.head())

In [26]:
df['Primary_Source'].unique()

array(['Thermal', 'Hydro', 'Geothermal', 'Photovoltaic',
       'Self-consumption', 'Wind'], dtype=object)

In [28]:
df

,Date,Actual_Generation_GWh,Primary_Source
0,2018-03-02 23:00:00,14.92,Thermal
1,2018-03-02 23:00:00,3.65,Hydro
2,2018-03-02 23:00:00,0.66,Geothermal
3,2018-03-02 23:00:00,0,Photovoltaic
4,2018-03-02 23:00:00,2.33,Self-consumption
...,...,...,...
8791,2022-10-07 00:00:00,16.25,Thermal
8792,2022-10-07 00:00:00,1.31,Hydro
8793,2022-10-07 00:00:00,2.512,Self-consumption
8794,2022-10-07 00:00:00,0.62,Geothermal


In [27]:
#export to csv
df.to_csv('../Data/italy_data.csv', index=False)